In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import time

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import skimage.io
#from tensorflow.python.keras.applications import ResNet50
#from tensorflow.python.keras.models import Sequential
#from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions



In [3]:
import cv2

In [4]:
from tensorflow.keras.models import model_from_json, load_model

In [ ]:
#os.listdir('../input/prostate-cancer-grade-assessment')

In [ ]:
#os.listdir('../input/panda-model-1')

In [ ]:
sample = pd.read_csv('../input/prostate-cancer-grade-assessment/sample_submission.csv')

In [ ]:
model1 = load_model('../input/panda-model-1/panda_very_simple_net_based_on_resnet50.h5')

In [ ]:
from_fold = '../input/prostate-cancer-grade-assessment/test_images'

In [ ]:
def prepare_test_data(test_arr):
    test_data = []
    images_names = []
    for test_name in test_arr:        
        image_id = test_name.split('.')[0]        
        test_file = os.path.join(from_fold, test_name)
        biopsy = skimage.io.MultiImage(test_file)
        out_file = cv2.resize(biopsy[-1], (512, 512))
        test_data.append(out_file)
        images_names.append(image_id)
        #print(time.time())
        del biopsy
    return np.array(images_names), np.array(test_data)

In [ ]:
def make_submission_df(id_array, test_labels):
    #assert len(id_array) == len(test_labels), "different length for label arrays"
    #assert isinstance(id_array, np.ndarray), 'images id is not a numpy array'
    #assert isinstance(test_labels, np.ndarray), 'test labels array is not a numpy array'
    d2 = {'image_id': id_array, 'isup_grade': test_labels}
    new_sample = pd.DataFrame(data=d2)
    #final_sample = pd.concat([sample_df, new_sample], ignore_index=True)
    return new_sample #final_sample

In [ ]:
#test_files = os.listdir(from_fold)[:10]
#test_files

In [ ]:
#biopsy = skimage.io.MultiImage(os.path.join(from_fold,'0cc35bc0fe4dd912b20f72d66888fd49.tiff'))

In [ ]:
#biopsy[-1].shape

In [ ]:
if os.path.exists(from_fold):
    test_files = os.listdir(from_fold) #[:100]
    %time test_files_id_array,  test_file_data_array = prepare_test_data(test_files)
    print(test_files_id_array.shape)
    print(test_file_data_array.shape)
    %time pred_all = model1.predict(test_file_data_array)
    final_test_labels = pred_all.argmax(axis=-1)
    submission = make_submission_df(test_files_id_array, final_test_labels)
    submission['isup_grade'] = submission['isup_grade'].astype(int)
    submission.head()
    submission.to_csv('submission.csv', index=False)
else:
    sub = pd.read_csv("/kaggle/input/prostate-cancer-grade-assessment/sample_submission.csv")
    sub.to_csv('submission.csv', index=False)
    print('no submission data')